# Twitter Sentiment Analysis

## Connect to Twitter API

In [408]:
#!pip install python-twitter
import twitter
import pandas as pd

API_KEY = "4QlG7dBQ1UeHAXyk1pC6n7LTm"
API_KEY_SECRET = "td6zhDHTI795WKXGYjXBJUwba9xCyxZ3o73VYro9XtLrQD30UZ"
ACCESS_TOKEN = "1067118225744371712-8rjdKc3f2rFNf7FXczj5T3X1W6yfu4"
ACCESS_TOKEN_SECRET = "Hw1A7nii1S2JAh0hMgkR6xZB1IddC1KAgJBPI5OaoQnSv"

# initialize api instance
twitter_api = twitter.Api(consumer_key=API_KEY,
                  consumer_secret=API_KEY_SECRET,
                  access_token_key=ACCESS_TOKEN,
                  access_token_secret=ACCESS_TOKEN_SECRET)

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Mon Nov 26 18:09:45 +0000 2018", "default_profile": true, "default_profile_image": true, "friends_count": 2, "id": 1067118225744371712, "id_str": "1067118225744371712", "name": "EliskaKloberdanz", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "EliskaKloberda1"}


## Search Twitter for any key word for which you want to analyze sentiment

In [417]:
# serach tweets by key words
def buildTestSet(search_keyword):
    text = []
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count=100)
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        for status in tweets_fetched:
            text.append(status.text)
        return pd.DataFrame({'tweet_text': text})
        #return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [418]:
search_term = input("Enter a search keyword: ")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword: oil
Fetched 100 tweets for the term oil
                                          tweet_text
0  India is the only country in the world where G...
1  Argentina has circulated a proposal to set $45...
2  The leaders of the Green Party and the Bloc Qu...
3  RT @SmilyRose1994: He Trying To Anal Fingering...


In [ ]:
testDataSet

## Build Train Set

In [ ]:
corpus = pd.read_csv('labeled_corpus.csv')
corpus.columns = ['', 'topic', 'label', 'tweet_id']
corpus = corpus.drop(corpus.columns[0], axis=1)
corpus.head(6)

In [72]:
import pandas as pd
import csv
import time 
def downloadTweetText(corpusFile):
    corpus = pd.read_csv(corpusFile)
    corpus.columns = ['', 'topic', 'label', 'tweet_id']
    corpus = corpus.drop(corpus.columns[0], axis=1)
    rate_limit=180
    sleep_time=900/180
    
    training_tweets_text =[]

    for tweet in corpus['tweet_id']:
        try:
            status = twitter_api.GetStatus(tweet)
            print("Tweet fetched" + status.text)
            training_tweets_text.append(status.text)
            #time.sleep(sleep_time)
        except: 
            training_tweets_text.append('NaN')
            continue
    print("Finished scraping")
    return training_tweets_text

In [ ]:
corpusFile = 'labeled_corpus.csv'
trainingDataTweetText = downloadTweetText(corpusFile)

In [74]:
unique = list(set(trainingDataTweetText))
trainingData = pd.read_csv(corpusFile)
trainingData['tweet_text'] = trainingDataTweetText
corpus.columns = ['', 'topic', 'label', 'tweet_id', 'tweet_text']
trainingData.to_csv('trainingData.csv')

In [423]:
trainingData = pd.read_csv('trainingData.csv')

## Data Pre-processing

In [424]:
# Drop NaNs
trainingData = trainingData[trainingData['tweet_text'].isnull() == False]

# Unique Labels
trainingData['label'].unique()

# Encode Labels
trainingData.loc[trainingData.label == 'neutral', 'label'] = 0
trainingData.loc[trainingData.label == 'irrelevant', 'label'] = 2
trainingData.loc[trainingData.label == 'positive','label'] = 1
trainingData.loc[trainingData.label == 'negative', 'label'] = -1

# Check dirstibution of labels
trainingData.groupby('label').size()


label
-1    253
 0    248
 1    116
dtype: int64

In [425]:
trainingData.head(5)

,topic,label,tweet_id,tweet_text
1,apple,1,1.264028e+17,Hilarious @youtube video - guy does a duet wit...
2,apple,1,1.263972e+17,@RIM you made it too easy for me to switch to ...
5,apple,1,1.263797e+17,The 16 strangest things Siri has said so far. ...
6,apple,1,1.263777e+17,Great up close & personal event @Apple tonight...
7,apple,1,1.263738e+17,From which companies do you experience the bes...


In [426]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_Text(data):
    words = set(nltk.corpus.words.words())
    for row in data.index:
        text = data.loc[row, 'tweet_text']
        text = text.lower()
        text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w in words and w.isalpha()) #remove non-english words
        text = re.sub('<[^<]+?>', '', text) #remove html tags
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) #remove links
        text = text.strip('=')
        text = text.split(" ") 
        text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
        lem = nltk.stem.WordNetLemmatizer()
        text = [lem.lemmatize(i) for i in text]
        text = ' '.join(str(e) for e in text)
        data.loc[row,'tweet_text'] = text #replace description in place in the dataframe
        row += 1
    return data

[nltk_data] Downloading package words to /home/eliska/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eliska/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eliska/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [429]:
trainingData = preprocess_Text(trainingData)
trainingData.head(5)

,topic,label,tweet_id,tweet_text
1,apple,1,1.264028e+17,hilarious video guy duet apple pretty much lov...
2,apple,1,1.263972e+17,rim made easy switch apple see ya
5,apple,1,1.263797e+17,said far glad apple gave sense humor via
6,apple,1,1.263777e+17,great close personal event apple tonight regen...
7,apple,1,1.263738e+17,experience best customer service aside apple


In [428]:
testingData = preprocess_Text(testDataSet)
testingData.head(5)

,tweet_text
0,country world increasing price petrol diesel d...
1,proposal set per barrel price oil bid keep
2,green party bloc calling government stop suppo...
3,trying anal fingering oil massage watch full v...
4,


## Vectorization

In [437]:
# Create our vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(list(train_set['tweet_text']))

# Build the classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=.01)



Total accuracy classification score: 0.6290322580645161
Total F1 classification score: 0.6034058656575213


## Split data intro train and test sets

In [436]:
# Split data into traning and testing
split_ratio = 0.7
total_number_of_datapoints = len(trainingData.index)
train_number_of_datapoints = int(total_number_of_datapoints * split_ratio)

from sklearn.utils import shuffle
trainingData = shuffle(trainingData)

train_set = trainingData.iloc[:train_number_of_datapoints, :]
test_set = trainingData.iloc[train_number_of_datapoints:, :]



## Model

In [ ]:
#  Train the classifier
clf.fit(vectors, train_set['label'].values)

# Get the test vectors
vectors_test = vectorizer.transform(list(test_set['tweet_text']))

# Predict and score the vectors
from sklearn import metrics
pred = clf.predict(vectors_test)
acc_score = metrics.accuracy_score( test_set['label'].values, pred)
f1_score = metrics.f1_score( test_set['label'].values, pred, average='macro')

print('Total accuracy classification score: {}'.format(acc_score))
print('Total F1 classification score: {}'.format(f1_score))



## Apply model

In [440]:
# Get the test vectors
vectors_test = vectorizer.transform(list(testDataSet['tweet_text']))
pred = clf.predict(vectors_test)

In [445]:
testDataSet['label'] = pred

In [443]:
# Decode Labels
testDataSet.loc[testDataSet.label == 0, 'label'] = 'neutral'
testDataSet.loc[testDataSet.label == 2, 'label'] = 'irrelevant'
testDataSet.loc[testDataSet.label == 1,'label'] = 'positive'
testDataSet.loc[testDataSet.label == -1, 'label'] = 'negative'

In [444]:
testDataSet.head(6)

,tweet_text,label
0,country world increasing price petrol diesel d...,neutral
1,proposal set per barrel price oil bid keep,positive
2,green party bloc calling government stop suppo...,negative
3,trying anal fingering oil massage watch full v...,neutral
4,,negative
5,smart really pinnacle bygone era utopian born ...,negative
